In [45]:
import pandas as pd # Import pandas for data manipulation
import haversine as hs  # Import haversine for calculating distances between two points
from haversine import Unit  # Import Unit to specify the distance unit for haversine

In [46]:
# Load the Uber dataset
df=pd.read_csv("uber.csv")

In [47]:
# Drop unnecessary columns 'tt' and 'key'
df=df.drop(columns=['tt'])
df=df.drop(columns=['key'])
# Convert 'pickup_datetime' column to datetime format
df['pickup_datetime']=pd.to_datetime(df['pickup_datetime'])
# Remove duplicate rows and reset index
df=df.drop_duplicates().reset_index(drop=True)

In [48]:
#Filter out rows where latitude or longitude are out of bounds
df = df[
    (df['pickup_latitude'].between(-90, 90)) &
    (df['pickup_longitude'].between(-180, 180)) &
    (df['dropoff_latitude'].between(-90, 90)) &
    (df['dropoff_longitude'].between(-180, 180))
]
# Filter out invalid passenger counts (e.g., incorrect value of 208 or 0)
df = df[ (df['passenger_count']!= 208) &
    (df['passenger_count']!= 0) ]

df

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1
1,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1
2,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1
3,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3
4,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5
...,...,...,...,...,...,...,...
199995,3.0,2012-10-28 10:49:00+00:00,-73.987042,40.739367,-73.986525,40.740297,1
199996,7.5,2014-03-14 01:09:00+00:00,-73.984722,40.736837,-74.006672,40.739620,1
199997,30.9,2009-06-29 00:42:00+00:00,-73.986017,40.756487,-73.858957,40.692588,2
199998,14.5,2015-05-20 14:56:25+00:00,-73.997124,40.725452,-73.983215,40.695415,1


In [49]:
# --- Dimensional Tables Creation ---

# Create time dimension table based on 'pickup_datetime'

Dim_time=df[['pickup_datetime']].reset_index(drop=True)
Dim_time['Time_key']=Dim_time.index+1
Dim_time['pickup_datetime']=Dim_time['pickup_datetime']
Dim_time['Pickup_weekday']=Dim_time['pickup_datetime'].dt.day_name()
Dim_time['Pickup_time']=Dim_time['pickup_datetime'].dt.time
Dim_time['Pickup_hour']=Dim_time['pickup_datetime'].dt.hour
Dim_time['Pickup_day']=Dim_time['pickup_datetime'].dt.day
Dim_time['Pickup_month']=Dim_time['pickup_datetime'].dt.month
Dim_time['Pickup_Year']=Dim_time['pickup_datetime'].dt.year

# Reorder columns in time dimension table
Dim_time=Dim_time[['Time_key','pickup_datetime','Pickup_weekday','Pickup_time','Pickup_hour','Pickup_day','Pickup_month','Pickup_Year']]

Dim_time

,Time_key,pickup_datetime,Pickup_weekday,Pickup_time,Pickup_hour,Pickup_day,Pickup_month,Pickup_Year
0,1,2015-05-07 19:52:06+00:00,Thursday,19:52:06,19,7,5,2015
1,2,2009-07-17 20:04:56+00:00,Friday,20:04:56,20,17,7,2009
2,3,2009-08-24 21:45:00+00:00,Monday,21:45:00,21,24,8,2009
3,4,2009-06-26 08:22:21+00:00,Friday,08:22:21,8,26,6,2009
4,5,2014-08-28 17:47:00+00:00,Thursday,17:47:00,17,28,8,2014
...,...,...,...,...,...,...,...,...
199273,199274,2012-10-28 10:49:00+00:00,Sunday,10:49:00,10,28,10,2012
199274,199275,2014-03-14 01:09:00+00:00,Friday,01:09:00,1,14,3,2014
199275,199276,2009-06-29 00:42:00+00:00,Monday,00:42:00,0,29,6,2009
199276,199277,2015-05-20 14:56:25+00:00,Wednesday,14:56:25,14,20,5,2015


In [50]:
# Create pickup location dimension table
Dim_Loc_Pickup=df[['pickup_longitude','pickup_latitude']].reset_index(drop=True)
Dim_Loc_Pickup['LocPick_id']=Dim_Loc_Pickup.index+1 # Add unique ID for each pickup location

Dim_Loc_Pickup=Dim_Loc_Pickup[['LocPick_id','pickup_longitude','pickup_latitude']]

Dim_Loc_Pickup

,LocPick_id,pickup_longitude,pickup_latitude
0,1,-73.999817,40.738354
1,2,-73.994355,40.728225
2,3,-74.005043,40.740770
3,4,-73.976124,40.790844
4,5,-73.925023,40.744085
...,...,...,...
199273,199274,-73.987042,40.739367
199274,199275,-73.984722,40.736837
199275,199276,-73.986017,40.756487
199276,199277,-73.997124,40.725452


In [51]:
# Create dropoff location dimension table
Dim_Loc_dropoff=df[['dropoff_longitude','dropoff_latitude']].reset_index(drop=True)
Dim_Loc_dropoff['LocDrop_id']=Dim_Loc_dropoff.index+1 # Add unique ID for each dropoff location

Dim_Loc_dropoff=Dim_Loc_dropoff[['LocDrop_id','dropoff_longitude','dropoff_latitude']]

Dim_Loc_dropoff

,LocDrop_id,dropoff_longitude,dropoff_latitude
0,1,-73.999512,40.723217
1,2,-73.994710,40.750325
2,3,-73.962565,40.772647
3,4,-73.965316,40.803349
4,5,-73.973082,40.761247
...,...,...,...
199273,199274,-73.986525,40.740297
199274,199275,-74.006672,40.739620
199275,199276,-73.858957,40.692588
199276,199277,-73.983215,40.695415


In [52]:

# Convert coordinates to numeric, coercing invalid values to NaN
df['pickup_latitude'] = pd.to_numeric(df['pickup_latitude'], errors='coerce')
df['pickup_longitude'] = pd.to_numeric(df['pickup_longitude'], errors='coerce')
df['dropoff_latitude'] = pd.to_numeric(df['dropoff_latitude'], errors='coerce')
df['dropoff_longitude'] = pd.to_numeric(df['dropoff_longitude'], errors='coerce')

# Calculate distance between pickup and dropoff locations using haversine formula (in kilometers)
df['distance'] = df.apply(lambda row:round( hs.haversine(
    (row['pickup_latitude'], row['pickup_longitude']),
    (row['dropoff_latitude'], row['dropoff_longitude']),
    unit=Unit.KILOMETERS),2), axis=1)

# Create distance dimension table
Dim_Distance=df[['distance']].reset_index(drop=True)
Dim_Distance['Ride_distance']=Dim_Distance['distance'] # Rename column
Dim_Distance['Distance_id']=Dim_Distance.index+1 # Add unique distance ID

Dim_Distance=Dim_Distance[['Distance_id','Ride_distance']]

Dim_Distance


,Distance_id,Ride_distance
0,1,1.68
1,2,2.46
2,3,5.04
3,4,1.66
4,5,4.48
...,...,...
199273,199274,0.11
199274,199275,1.88
199275,199276,12.85
199276,199277,3.54


In [53]:
# Calculate fare rate per ride (fare_amount divided by distance)
df['RateRide_code']=df['fare_amount']/df['distance']

# Create ride fare categories based on rate quartiles
df['RateRide_code'].quantile([0.25, 0.5, 0.75]) # Calculate quartiles

df.loc[df['RateRide_code'] < 2.988024, 'RateRide_code'] = 1 # Low Fare
df.loc[df['RateRide_code'].between(2.988024, 5.209104), 'RateRide_code'] = 2 # Average Fare
df.loc[df['RateRide_code'] > 5.209104, 'RateRide_code'] = 3 # High Fare


# Create rate ride dimension table
Dim_RateRide=df[['RateRide_code']].reset_index(drop=True)
Dim_RateRide['RateRide_id']=Dim_RateRide.index+1 # Add unique ID for fare rates
# Label fares
Dim_RateRide.loc[Dim_RateRide['RateRide_code'] == 1, 'RateRide_codeName'] = 'Low Fare'
Dim_RateRide.loc[Dim_RateRide['RateRide_code'] == 2, 'RateRide_codeName'] = 'Avg Fare' 
Dim_RateRide.loc[Dim_RateRide['RateRide_code'] == 3, 'RateRide_codeName'] = 'High Fare'

Dim_RateRide= Dim_RateRide[['RateRide_id','RateRide_code','RateRide_codeName']]

Dim_RateRide

0          4.464286
1          3.130081
2          2.559524
3          3.192771
4          3.571429
            ...    
199995    27.272727
199996     3.989362
199997     2.404669
199998     4.096045
199999     2.601476
Name: RateRide_code, Length: 199278, dtype: float64


,RateRide_id,RateRide_code,RateRide_codeName
0,1,2.0,Avg Fare
1,2,2.0,Avg Fare
2,3,1.0,Low Fare
3,4,2.0,Avg Fare
4,5,2.0,Avg Fare
...,...,...,...
199273,199274,3.0,High Fare
199274,199275,2.0,Avg Fare
199275,199276,1.0,Low Fare
199276,199277,2.0,Avg Fare


In [54]:
# Create fact table that combines fare and ride information
Fact_Fare=df[['fare_amount', 'passenger_count']].reset_index(drop=True)
Fact_Fare['VendorID']=Fact_Fare.index+1 # Add unique ID
Fact_Fare['Time_key']=Dim_time['Time_key'] 
Fact_Fare['LocPick_id']=Dim_Loc_Pickup['LocPick_id']
Fact_Fare['LocDrop_id']=Dim_Loc_dropoff['LocDrop_id']
Fact_Fare['Distance_id']=Dim_Distance['Distance_id']
Fact_Fare['RateRide_id']=Dim_RateRide['RateRide_id']

Fact_Fare = Fact_Fare[['VendorID', 'Time_key', 'LocPick_id', 'LocDrop_id', 'Distance_id', 'RateRide_id', 'fare_amount', 'passenger_count']]
Fact_Fare.head()

,VendorID,Time_key,LocPick_id,LocDrop_id,Distance_id,RateRide_id,fare_amount,passenger_count
0,1,1,1,1,1,1,7.5,1
1,2,2,2,2,2,2,7.7,1
2,3,3,3,3,3,3,12.9,1
3,4,4,4,4,4,4,5.3,3
4,5,5,5,5,5,5,16.0,5


In [56]:
# --- Data Cleanup: Remove Invalid Data Rows ---

# 1. Filter out rows with distance == 0 from distance_table
rows_to_delete = Dim_Distance[Dim_Distance['Ride_distance'] == 0]['Distance_id']

# 2. Delete the rows in distance_table
Dim_Distance = Dim_Distance[Dim_Distance['Ride_distance'] != 0]

# 3. Remove corresponding rows in other tables based on 'ID'
Dim_time = Dim_time[~Dim_time['Time_key'].isin(rows_to_delete)]
Dim_RateRide = Dim_RateRide[~Dim_RateRide['RateRide_id'].isin(rows_to_delete)]
Dim_Loc_dropoff = Dim_Loc_dropoff[~Dim_Loc_dropoff['LocDrop_id'].isin(rows_to_delete)]
Dim_Loc_Pickup = Dim_Loc_Pickup[~Dim_Loc_Pickup['LocPick_id'].isin(rows_to_delete)]
Fact_Fare = Fact_Fare[~Fact_Fare['VendorID'].isin(rows_to_delete)]

# 4. Filter out rows with Fare Amount == 0 from Fact_table
rows_to_delete1 = Fact_Fare[Fact_Fare['fare_amount'] == 0]['VendorID']
# 5. Delete the rows in distance_table
Fact_Fare = Fact_Fare[Fact_Fare['fare_amount'] != 0]

# 6. Remove corresponding rows in other tables based on 'ID'
Dim_time = Dim_time[~Dim_time['Time_key'].isin(rows_to_delete1)]
Dim_RateRide = Dim_RateRide[~Dim_RateRide['RateRide_id'].isin(rows_to_delete1)]
Dim_Loc_dropoff = Dim_Loc_dropoff[~Dim_Loc_dropoff['LocDrop_id'].isin(rows_to_delete1)]
Dim_Loc_Pickup = Dim_Loc_Pickup[~Dim_Loc_Pickup['LocPick_id'].isin(rows_to_delete1)]
Fact_Fare = Fact_Fare[~Fact_Fare['VendorID'].isin(rows_to_delete1)]

# 7. Reset the 'index' column for all tables
Dim_Distance.reset_index(drop=True, inplace=True)
Dim_time.reset_index(drop=True, inplace=True)
Dim_RateRide.reset_index(drop=True, inplace=True)
Dim_Loc_dropoff.reset_index(drop=True, inplace=True)
Dim_Loc_Pickup.reset_index(drop=True, inplace=True)
Fact_Fare.reset_index(drop=True, inplace=True)

# 8. Reset the 'ID' column for all tables
Dim_Distance['Distance_id'] = range(1, len(Dim_Distance) + 1)
Dim_time['Time_key'] = range(1, len(Dim_time) + 1)
Dim_RateRide['RateRide_id'] = range(1, len(Dim_RateRide) + 1)
Dim_Loc_dropoff['LocDrop_id'] = range(1, len(Dim_Loc_dropoff) + 1)
Dim_Loc_Pickup['LocPick_id'] = range(1, len(Dim_Loc_Pickup) + 1)
Fact_Fare['VendorID'] = range(1, len(Fact_Fare) + 1)

C:\Users\kerol\AppData\Local\Temp\ipykernel_77832\1887730182.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dim_Distance['Distance_id'] = range(1, len(Dim_Distance) + 1)


In [58]:
# --- Export the cleaned and transformed data to CSV files ---
Dim_time.to_csv("Dim_time.csv", index=False)
Dim_RateRide.to_csv("Dim_RateRide.csv", index=False)
Dim_Distance.to_csv("Dim_Distance.csv", index=False)
Dim_Loc_dropoff.to_csv("Dim_Loc_dropoff.csv", index=False)
Dim_Loc_Pickup.to_csv("Dim_Loc_Pickup.csv", index=False)
Fact_Fare.to_csv("FactFare.csv", index=False)


In [59]:
# --- Export the cleaned data to Excel files ---
Dim_RateRide.to_excel("Dim_RateRide.xlsx", index=False)
Dim_Distance.to_excel("Dim_Distance.xlsx", index=False)
Dim_Loc_dropoff.to_excel("Dim_Loc_dropoff.xlsx", index=False)
Dim_Loc_Pickup.to_excel("Dim_Loc_Pickup.xlsx", index=False)
Fact_Fare.to_excel("FactFare.xlsx", index=False)

# Remove timezone information from 'pickup_datetime' column
Dim_time['pickup_datetime'] = Dim_time['pickup_datetime'].dt.tz_localize(None)

# Export the time dimension table to Excel
Dim_time.to_excel("Dim_time.xlsx", index=False)